In [9]:
import pandas as pd
import numpy as np
import random
from functools import reduce

In [10]:
df = pd.read_csv("info.csv").iloc[:,1:]
df = df.set_index("Your name")

In [73]:
df.head()

,First Person,Second person,Third person,Fourth person,Fifth person
Your name,,,,,
1,13.0,12.0,78.0,8.0,34.0
2,61.0,42.0,39.0,3.0,69.0
3,61.0,39.0,2.0,69.0,44.0
4,37.0,21.0,14.0,28.0,51.0
5,43.0,7.0,30.0,41.0,32.0


In [216]:
class Evaluation:  
    def VerticleHappiness(self, people):
        HappyPeople = []
        for person in people:
            row, col = self.getRowColFromPerson(person)

            choices = self.getChoices(person)

            if row!=self.numRows-1:
                p = self.Chart[row+1][col]
                rank = self.rank(p,choices)

                if rank in range(1,6) and person not in HappyPeople:
                    HappyPeople.append(person)


            if row!=0:
                p = self.Chart[row-1][col]

                rank = self.rank(p,choices)
 
                if rank in range(1,6) and person not in HappyPeople:
                    HappyPeople.append(person)
        return HappyPeople
            
    def rank(self,p,choices):
        if p in choices: return choices.index(p)+1
        return None
    
    def EvaluateHappiness(self):
        Score = {}
        Done = []
        for Level in range(1,6):
            
            HappyPeople = []
            NoPreferences = []
            UnhappyPeople = []
            
            for row in range(self.numRows):
                for col in range(self.numCols):
                    person = self.Chart[row][col]
                    if person==0: continue
                
                    if self.NaPerson(person): 
                        NoPreferences.append(person)
                        continue

                    choices = self.getChoices(person)
                    
                    if col!=self.numCols-1:
                        p = self.Chart[row][col+1]
                        
                        rank = self.rank(p,choices)
                        if rank == Level and person not in HappyPeople and person not in Done: 
                            HappyPeople.append(person)
                        
                        
                    if col!=0:
                        p = self.Chart[row][col-1]
                        rank = self.rank(p,choices)
                        
                        if rank == Level and person not in HappyPeople and person not in Done:
                            HappyPeople.append(person)
                        
            Score[Level] = HappyPeople
            Done += HappyPeople
        
        
        numUnhappy = 94-len(NoPreferences)-len(Done)
        UnhappyPeople = [p for p in range(1,95) if p not in NoPreferences+Done]
        HaveVerticle = self.VerticleHappiness(UnhappyPeople)
        numVerticleHappy = len(HaveVerticle)
        
#         for level in range(1,6):
#             print("Level %s, # of people %s" %(level, len(Score[level])))
#         print("# of people No Preferences", len(NoPreferences))
#         print("Not in Top 5:", numUnhappy)
#         print("Not in Top 5:", UnhappyPeople)
#         print("Number of People with verticle: %s "%HaveVerticle)
        
        solution = FinalArrangement(self.Chart,len(Score[1]), UnhappyPeople, HaveVerticle)
        return solution
        
    
class FinalArrangement:
    def __init__(self,chart,numRankOne,UnhappyPeople,verticleHappy):
        self.Chart = chart
        self.numRankOne = numRankOne
        self.UnhappyPeople = UnhappyPeople
        self.numUnhappy = len(self.UnhappyPeople)
        self.VerticleHappy = verticleHappy
    def Print(self):
        print(self.Chart)
        print("Side by Side: First Choice:", self.numRankOne)
        print("Not in Top 5:", self.UnhappyPeople)
        print("Verticle Happy:", self.VerticleHappy)
        
class Solutions:
    def notSame(self, s,solution):
        return s.numRankOne != solution.numRankOne or s.numUnhappy != solution.numUnhappy
    def bestSolutions(self,solutions):
        k = solutions.copy()
        for solution in k:
            if list(filter(lambda s: s.numRankOne >= solution.numRankOne and s.numUnhappy <= solution.numUnhappy and self.notSame(s,solution), solutions)) != []:
                solutions.remove(solution)

        return solutions

    def printBestSolutions(self):
        for solution in self.solutions:
            solution.Print()
    

In [217]:
class FunctionTools:
    def getRowColFromPerson(self,person):
        for row in range(self.numRows):
                for col in range(self.numCols):
                    p = self.Chart[row][col]
                    
                    if person==p:
                        
                        return row,col
                    
    def idealPairs(self):
        self.firsts = []
        for pair in self.matches:
            if pair["rank1"] in [1,2,None] and pair["rank2"] in [1,2,None]:
                self.firsts.append(pair)
    
    def makeDict(self, person1, rank1, person2, rank2):
        return {"person1":person1,"rank1":rank1,"person2":person2, "rank2":rank2}
    
    def getChoices(self,p):
        return list(self.df.loc[p])
    
    def removePersonFromMatches(self,person):
        return [pair for pair in matches if not (pair["person1"]==person or pair["person2"]==person)]
    
    def removePersonFromPeople(self,p, people):
        return [person for person in people if person != p]
        
                
    def getMatches(self,person):
        return [pair for pair in self.matches if pair["person1"]==person or pair["person2"]==person]
    
    
    def NaPerson(self,person):
        choices = self.getChoices(person)
        return np.isnan(choices).all()         
    
    def getPeople(self,pair):
        return [pair["person1"],pair["person2"]]
    
    def CompositeScore(self, pair):
        rank1 = pair["rank1"]
        rank2 = pair["rank2"]
        totalRank = rank2*2 if rank1 == None else (rank1*2 if rank2==None else rank1+rank2)
        return totalRank
    
    def findOtherPerson(self,pair, person):
        return pair["person1"] if person != pair["person1"] else pair["person2"]
    
    def findMatchPeople(self,matchesLeft,personLeft, matchesAbove, personAbove):
        peopleLeft = [self.findOtherPerson(pair, personLeft) for pair in matchesLeft]
        peopleAbove = [self.findOtherPerson(pair, personAbove) for pair in matchesAbove]
        return list(set(peopleLeft).union(set(peopleAbove)))
    
    def samePeople(self,pair, p1,p2):
        np1, np2 = pair["person1"],pair["person2"]
        if (np1 == p1 and np2==p2) or (np2 == p1 and np1==p2):
            return True
        return False
    
    
    def getMatch(self,p1,p2):
        match = [pair for pair in self.matches if self.samePeople(pair,p1,p2)]
        if len(match)==0:return None
        return match[0]
    
    def scorePair(self, pair):
        rank1 = pair["rank1"]
        rank2 = pair["rank2"]
        totalRank = rank2*2 if rank1 == None else (rank1*2 if rank2==None else rank1+rank2)
        return totalRank
    


In [218]:

class Students(Evaluation, FunctionTools, Solutions):
    df = pd.read_csv("info.csv").iloc[:,1:]
    df = df.set_index("Your name")
    def __init__(self):
        self.numCols = 10
        self.numRows = 10
        self.matches()
        self.idealPairs()
        self.MatchesForLonelyPeople()
        self.stringPairs = []
        self.people = list(self.df.index)
        
        
        
        self.solutions = []
        for x in range(10):
            self.Chart = np.zeros((self.numRows,self.numCols))
            self.createString(0,0, self.people)
        self.solutions = self.bestSolutions(self.solutions)
        self.printBestSolutions()
        
        
   
    def matches(self):
        matches = []
        for person,rankings in self.df.iterrows():
                for rank1,rankedPerson in enumerate(rankings):
                    if not pd.isnull(rankedPerson):
                        OtherPersonChoices = self.getChoices(rankedPerson)
                        if person in OtherPersonChoices:
                            if person > rankedPerson: continue #don't want duplicates if they overlap it will be added by first person
                            rank2 = OtherPersonChoices.index(person)
                            matches.append(self.makeDict(person,rank1+1,rankedPerson,rank2))
                        elif np.isnan(OtherPersonChoices).all():
                            matches.append(self.makeDict(person,rank1+1,rankedPerson,None))
        self.matches = matches
    
    def NoMatches(self):
        haveMatch = reduce(lambda a,b: a+b, [self.getPeople(pair) for pair in self.matches])
        return [p for p in range(1,95) if p not in haveMatch and not self.NaPerson(p)]
    
    def MatchesForLonelyPeople(self):
        needMatch = self.NoMatches()
        for person in needMatch:
            rankings = self.getChoices(person)
            for rank1,rankedPerson in enumerate(rankings):
                if not pd.isnull(rankedPerson):
                    self.matches.append(self.makeDict(person,rank1+1,rankedPerson,rank1+3))
                    
    def CompositeScore(self,person,personLeft, personAbove):
        if len(self.getMatches(person))==1: return -100
        matchLeft = self.getMatch(person,personLeft)
        matchAbove = self.getMatch(person,personAbove)
        
        if matchAbove == None:
            totalRankLeft = self.scorePair(matchLeft)
            return totalRankLeft
        if matchLeft == None:
            totalRankAbove = self.scorePair(matchAbove)
            return 100+totalRankAbove
        
        totalRankLeft = self.scorePair(matchLeft)
        totalRankAbove = 12-self.scorePair(matchAbove)
        return totalRankLeft-.5*(totalRankAbove)
        
    def lowestCompositeScore(self, row,col, people):
        
        personLeft = 0 if col==0 else self.Chart[row][col-1]
        personAbove = 0 if row==0 else self.Chart[row-1][col]
        if (personLeft==0 and personAbove==0): return None
        matchesLeft = self.getMatches(personLeft)
        matchesAbove = self.getMatches(personAbove)
 
        
        peopleOptions = self.findMatchPeople(matchesLeft,personLeft,matchesAbove, personAbove)
        peopleOptions = [person for person in peopleOptions if person in people]
        if len(peopleOptions)==0: return None
        scores = [self.CompositeScore(person, personLeft, personAbove) for person in peopleOptions]
        minScore = min(scores)
        bestPerson = peopleOptions[scores.index(minScore)]
        return bestPerson
        
    
    
    def newPosition(self,row,col):
        if col==self.numCols-1: 
            col =0
            row += 1
        else: col+=1
        return row,col
    def setLeftToZero(self,row,col):
        row,col = self.newPosition(row,col)
        while row!=self.numRows:
            self.Chart[row][col]=0
            row,col = self.newPosition(row,col)
            
    
    def createString(self, row, col, people):

        if row == self.numRows: return
        newPerson = self.lowestCompositeScore(row, col, people)
        nrow,ncol = self.newPosition(row,col)
        if newPerson==None:
            if len(people)==0:
                #self.setLeftToZero(row,col)
                #print(self.Chart)
                solution = self.EvaluateHappiness()
                self.solutions.append(solution)
                return
            
            elif col==self.numCols-1:
                
                self.Chart[row][col] = 0
                self.createString(nrow, ncol, people)
                
            else: 
                
                #for x in people:
                #min(len(people),2)
                for newPerson in random.sample(people,1):
                    self.Chart[row][col] = newPerson
                    newpeople = self.removePersonFromPeople(newPerson, people)
                    self.createString(nrow, ncol, newpeople)
                

        else:  
        
            self.Chart[row][col] = newPerson
            newpeople = self.removePersonFromPeople(newPerson, people)

            self.createString(nrow, ncol, newpeople)
    
   
    
    
    

In [221]:
obj = Students()

[[60. 57. 40. 19. 25. 22. 53. 85. 52. 18.]
 [94. 62. 67. 68. 79. 90. 72. 59. 83. 20.]
 [82. 36. 43.  7. 42. 69. 39.  3. 61.  2.]
 [32. 41. 30.  5.  8. 78. 74. 14. 58. 81.]
 [49.  9. 65. 87. 29. 10. 84. 75. 76. 44.]
 [16. 45. 17. 48. 26. 47. 54. 56. 77. 88.]
 [93.  4. 37. 21. 38. 91. 80.  6. 23. 66.]
 [24. 63. 28. 51. 89. 70. 71. 46. 33. 92.]
 [ 1. 13. 12. 11. 27. 86. 31. 50. 73.  0.]
 [34. 15. 64. 55. 35.  0.  0.  0.  0.  0.]]
Side by Side: First Choice: 58
Not in Top 5: [33, 35, 46, 64, 66, 71]
Verticle Happy: [66, 71]
[[25. 19. 40. 57. 60. 37.  4. 21. 38. 91.]
 [22. 53. 85. 75. 84. 10. 29. 87. 65. 88.]
 [30. 41. 32. 82. 36. 43.  7. 42. 69. 39.]
 [33. 15. 34. 35. 86. 27. 31.  2. 61.  3.]
 [67. 68. 79. 90. 24. 63. 52. 76. 44. 64.]
 [62. 94. 71. 80. 54. 56. 77. 18. 73. 50.]
 [70. 89. 46.  6. 23. 51. 28.  8. 78. 74.]
 [20. 83. 59. 72. 93. 13.  1. 12. 11.  0.]
 [47. 26. 48. 45. 16. 17.  5. 14. 58. 81.]
 [49.  9. 92. 66. 55.  0.  0.  0.  0.  0.]]
Side by Side: First Choice: 57
Not in Top 5

In [215]:
obj = Students()

Side by Side: First Choice: 52
Not in Top 5: [6, 11, 17, 25]
Verticle Happy: [17]
Side by Side: First Choice: 56
Not in Top 5: [8, 20, 66]
Verticle Happy: [8, 66]
Side by Side: First Choice: 55
Not in Top 5: [6, 17, 71]
Verticle Happy: [6]
Side by Side: First Choice: 55
Not in Top 5: [5, 35, 46, 51, 71]
Verticle Happy: [5]
Side by Side: First Choice: 55
Not in Top 5: []
Verticle Happy: []
Side by Side: First Choice: 55
Not in Top 5: [6, 20, 35, 46]
Verticle Happy: [20, 46]
Side by Side: First Choice: 55
Not in Top 5: [6, 8, 20, 46, 72, 76]
Verticle Happy: [72]
Side by Side: First Choice: 55
Not in Top 5: [2, 6, 14, 71]
Verticle Happy: [2, 71]
Side by Side: First Choice: 54
Not in Top 5: [17, 20, 46, 71, 75]
Verticle Happy: [20, 75]
Side by Side: First Choice: 52
Not in Top 5: [6, 8, 35, 67]
Verticle Happy: [6, 35, 67]
[[ 9. 49. 16. 45. 17. 48. 26. 47. 57. 60.]
 [25. 19. 40. 53. 22. 85. 75. 84. 10. 29.]
 [50. 73. 44. 64. 86. 27. 31. 65. 87. 77.]
 [56. 54. 80. 68. 67. 33. 15. 34. 35. 88.

In [192]:
obj.solutions

In [225]:
df.loc[40]

First Person     57.0
Second person    19.0
Third person     25.0
Fourth person    22.0
Fifth person     53.0
Name: 40, dtype: float64

In [3]:
[[18. 52. 76. 44. 64. 26. 47. 67. 68. 79.]
 [ 2. 61.  3. 39. 69. 42.  7. 43.  5. 90.]
 [73. 50. 48. 45. 16. 17. 36. 46. 82. 32.]
 [53. 22. 57. 60. 49.  9. 58. 14. 38. 91.]
 [85. 75. 84. 10. 29. 87. 65. 88. 66. 92.]
 [25. 19. 40. 11. 12.  1. 13. 77. 56. 54.]
 [71. 80.  6. 23.  8. 78. 74. 93. 72. 59.]
 [37.  4. 21. 28. 51. 34. 15. 33. 89. 70.]
 [81. 30. 41. 31. 27. 86. 35. 20. 83.  0.]
 [24. 63. 62. 94. 55.  0.  0.  0.  0.  0.]]
Side by Side: First Choice: 55
Not in Top 5: []
Verticle Happy: []

1

In [ ]:
#go through every possible random

In [109]:
matches = []
for person,choices in df.iterrows():
        for rank1,choice in enumerate(choices):
            if person > choice: continue
            if not pd.isnull(choice):
                OtherPersonChoices = list(df.loc[choice])
                if person in OtherPersonChoices:
                    rank2 = OtherPersonChoices.index(person)
                    matches.append({"person1":person,"rank1":rank1+1,"person2":choice, "rank2":rank2+1})
                elif np.isnan(OtherPersonChoices).all():
                    matches.append({"person1":person,"rank1":rank1+1,"person2":choice, "rank2":None})
                
            

In [112]:
    def samePeople(self,pair, p1,p2):
        np1, np2 = pair["person1"],pair["person2"]
        if (np1 == p1 and np2==p2) or (np2 == p1 and np1==p2):
            return True
        return False
    
    
    def removeMatch(self,p1,p2, matches):
        return [pair for pair in matches if not self.samePeople(pair,p1,p2)]
    def verticle(self,row,col):
        verticle = []
        if row!=0:
            verticle.append(self.Chart[row-1][col])
        if row!=10:
            verticle.append(self.Chart[row+1][col])
        return verticle

[[1, 13.0],
 [1, 12.0],
 [3, 61.0],
 [3, 39.0],
 [4, 37.0],
 [4, 21.0],
 [5, 43.0],
 [7, 43.0],
 [7, 42.0],
 [8, 78.0],
 [9, 49.0],
 [10, 84.0],
 [10, 29.0],
 [14, 58.0],
 [15, 34.0],
 [15, 33.0],
 [16, 45.0],
 [18, 52.0],
 [19, 25.0],
 [19, 40.0],
 [20, 83.0],
 [21, 38.0],
 [22, 53.0],
 [22, 57.0],
 [26, 48.0],
 [26, 64.0],
 [27, 86.0],
 [27, 31.0],
 [28, 51.0],
 [29, 87.0],
 [31, 86.0],
 [33, 34.0],
 [36, 82.0],
 [38, 51.0],
 [40, 57.0],
 [42, 69.0],
 [44, 64.0],
 [45, 48.0],
 [50, 73.0],
 [52, 76.0],
 [53, 85.0],
 [54, 56.0],
 [54, 80.0],
 [58, 81.0],
 [59, 83.0],
 [59, 72.0],
 [65, 87.0],
 [65, 84.0],
 [66, 88.0],
 [67, 68.0],
 [68, 79.0],
 [70, 89.0],
 [72, 83.0],
 [75, 84.0],
 [77, 88.0]]

In [97]:
np.isnan([1,2,np.nan]).all()

False

In [82]:
df.loc[13]

First Person      1.0
Second person    78.0
Third person     12.0
Fourth person    34.0
Fifth person     15.0
Name: 13, dtype: float64

In [74]:
matches

[{'person1': 1, 'rank1': 1, 'person2': 13.0, 'rank2': 1},
 {'person1': 1, 'rank1': 2, 'person2': 12.0, 'rank2': 1},
 {'person1': 1, 'rank1': 3, 'person2': 78.0, 'rank2': 2},
 {'person1': 1, 'rank1': 4, 'person2': 8.0, 'rank2': 3},
 {'person1': 1, 'rank1': 5, 'person2': 34.0, 'rank2': 3},
 {'person1': 2, 'rank1': 1, 'person2': 61.0, 'rank2': 3},
 {'person1': 2, 'rank1': 2, 'person2': 42.0, 'rank2': 5},
 {'person1': 2, 'rank1': 3, 'person2': 39.0, 'rank2': 2},
 {'person1': 2, 'rank1': 4, 'person2': 3.0, 'rank2': 3},
 {'person1': 2, 'rank1': 5, 'person2': 69.0, 'rank2': 4},
 {'person1': 3, 'rank1': 1, 'person2': 61.0, 'rank2': 1},
 {'person1': 3, 'rank1': 2, 'person2': 39.0, 'rank2': 1},
 {'person1': 3, 'rank1': 4, 'person2': 69.0, 'rank2': 5},
 {'person1': 4, 'rank1': 1, 'person2': 37.0, 'rank2': 2},
 {'person1': 4, 'rank1': 2, 'person2': 21.0, 'rank2': 2},
 {'person1': 4, 'rank1': 3, 'person2': 14.0, 'rank2': 5},
 {'person1': 4, 'rank1': 4, 'person2': 28.0, 'rank2': 4},
 {'person1': 4, 